In [1]:
import rasterio
from rasterio.mask import mask
from rasterio import Affine
import geopandas as gpd
from shapely.geometry import box
import rasterio.plot as rplot
import matplotlib.pyplot as plt
import fiona
from shapely.geometry import box
from fiona.crs import from_epsg

# use trait image as extent and clip brdf_corrected avirirs image directly

## the trait image has been projected into a same projection system as aviris image before 

In [2]:
raster1 = rasterio.open(r'F:\wenqu\Aviris\site2a\output\ang20170709t012605_corr_v2p9_img_brdf')
raster2 = rasterio.open(r'F:\wenqu\trait_map\trait_proj\site2c_pc_trait_project_to_aviris.tif')

In [3]:
# Define the extent of raster2 as a bounding box:
xmin, ymin, xmax, ymax = raster2.bounds
bbox = box(xmin, ymin, xmax, ymax)
print(bbox)

POLYGON ((623502.6151804522 7474143.746850188, 623502.6151804522 7474302.864936455, 623362.8689653493 7474302.864936455, 623362.8689653493 7474143.746850188, 623502.6151804522 7474143.746850188))


# write the bounding box into a shapefile

In [4]:
crs = from_epsg(raster2.crs.to_epsg())
# Define the output shapefile
output_file = 'F:/wenqu/Aviris/site2c/site2c_shp/site2c_bounding_box.shp'

# Write the bounding box polygon to a shapefile with the same CRS as the input raster
with fiona.open(output_file, 'w', driver='ESRI Shapefile', crs=crs, schema={'geometry': 'Polygon'}) as dst:
    dst.write({'geometry': bbox.__geo_interface__, 'properties': {}})

In [5]:
raster2.crs, raster1.crs

(CRS.from_epsg(32605), CRS.from_epsg(32605))

In [6]:
# Reproject the bounding box of raster2 to the coordinate system of raster1:
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0])
# set raster2 crs as the coordinates reference system 
geo = geo.set_crs(raster2.crs)

# reproject geo to the CRS of raster1
# geo = geo.to_crs(raster1.crs)

In [7]:
# Clip raster1 using the extent of raster2:
out_img, out_transform = mask(raster1, shapes=geo.geometry, crop=True)

# Update the metadata of the output image:
out_meta = raster1.meta.copy()
out_meta.update({
    "driver": "ENVI",
    "height": out_img.shape[1],
    "width": out_img.shape[2],
    "count":425,
    "transform": out_transform
})

In [9]:
output = "F:/wenqu/Aviris/site2c/site2c_aviris_clip/site2c_aviris_clip"
with rasterio.open(output, "w", **out_meta) as dst:
    # loop over the bands and write the pixel value for each band
    for band in range(1, raster1.count + 1):
        dst.write(out_img[band-1], indexes=band)